In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import gc
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
submission.head()

,id,label
0,0,5
1,1,5
2,2,5
3,3,5
4,4,5


In [12]:
test.head()

,id,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train.head(20)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
%%time
X = train[train.columns[1:]].values
Y = train.label.values

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

Wall time: 138 ms


(60000, 10)

In [6]:
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = LogisticRegression(C=15, solver='lbfgs', multi_class='multinomial', max_iter=80)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(test[test.columns[1:]].values)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1


C:\Users\anany\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold accuracy: 0.9620833333333333
Fitting fold 2


C:\Users\anany\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold accuracy: 0.96325
Fitting fold 3


...

Fold accuracy: 0.964
Fitting fold 4


C:\Users\anany\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold accuracy: 0.9624166666666667
Fitting fold 5


C:\Users\anany\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold accuracy: 0.9616666666666667


In [7]:
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

0.9626833333333333


In [8]:
preds = np.argmax(test_preds, axis=1)
submission['label'] = preds
submission.to_csv('submission.csv', index=False)
submission.head(20)

,id,label
0,0,3
1,1,0
2,2,2
3,3,6
4,4,7
5,5,7
6,6,1
7,7,8
8,8,7
9,9,4


In [9]:
np.save('test_preds', test_preds)
np.save('train_oof', train_oof)

In [10]:
test_preds

array([[3.44858222e-10, 8.31452400e-11, 1.04121400e-04, ...,
        2.77106637e-04, 5.65217805e-17, 6.84527796e-09],
       [9.99693361e-01, 1.27315372e-05, 2.50207743e-04, ...,
        1.34691078e-06, 2.08081827e-06, 1.67468605e-07],
       [9.66833798e-13, 1.86371535e-05, 9.99934356e-01, ...,
        1.20634043e-15, 7.98617691e-10, 2.23800238e-16],
       ...,
       [2.28965328e-05, 9.99977100e-01, 2.59628573e-11, ...,
        4.85247272e-11, 3.32981376e-12, 4.12094098e-13],
       [2.21814120e-10, 3.64831266e-12, 4.49346990e-16, ...,
        1.77392029e-02, 7.20997725e-12, 7.37200110e-11],
       [5.86549624e-07, 3.18963000e-08, 6.39718995e-03, ...,
        3.55643174e-01, 3.85102942e-12, 9.85678535e-04]])

In [11]:
train_oof

array([[9.99964426e-01, 3.55602134e-05, 8.04323223e-09, ...,
        3.63745522e-11, 6.31212312e-13, 1.11708319e-14],
       [1.26722867e-05, 9.99987326e-01, 3.39178577e-10, ...,
        3.58316409e-19, 1.43973541e-09, 2.14400192e-20],
       [1.29685660e-12, 2.32794990e-16, 1.00000000e+00, ...,
        9.00873337e-22, 5.99256883e-24, 4.57522289e-15],
       ...,
       [2.16577572e-09, 2.16803765e-09, 1.65929564e-15, ...,
        9.13539292e-01, 3.41601504e-20, 6.02326076e-10],
       [1.51490378e-14, 2.84956762e-21, 5.24993797e-20, ...,
        5.91166226e-25, 9.99999994e-01, 5.53466352e-13],
       [8.62921595e-11, 4.96043178e-09, 7.58025255e-08, ...,
        1.03809352e-16, 7.12616184e-10, 9.99999918e-01]])